
# Daedalus Coverage

## Introduction
The purpose of this software is to calculate how much time the Daedalus satellite will spend inside various
areas of interest. This knowledge enables us to adjust the orbit parameters so that the satellite visits 
these areas more. 
We call these areas 'bins'. A goal of total time of satellite presence is assigned to each bin.
The bin boundaries are defined by ranges of:
1. Magnetic Local Time
2. Magnetic Latitude
3. Altitude
4. Geomagnetic Kp index


## Data


### Orbit data
The satellite positions are described by Time, Latitide, Longitude and Altitude and are calculated 
by '....' software, with 10 seconds frequency for the entire duration of the mission (3 years). 
For each position the Magnetic Latitude and Magnetic Local Time are calculated, as well.
These data constitute one of the inputs for the current software.

### Geomagnetic Kp Indices

Geomagnetic Kp indices describe the variation in the geomagnetic field caused mainly by solar radiation changes and also by changes and interactions at the magnetosphere and the ionosphere.
Kp indices are calculated over a 3-hour period and range in 28 steps from 0 (quiet) to 9 (greatly disturbed) with fractional parts expressed in thirds of a unit.
We used Kp indices from previous years as they are recorded from NOAA web site:

  - Explanation: https://www.ngdc.noaa.gov/stp/GEOMAG/kp_ap.html
  - Download location: ftp://ftp.ngdc.noaa.gov/STP/GEOMAGNETIC_DATA/INDICES/KP_AP 
  
There are available data for the years 1932-2018. The user can select which of these years' values to use for calculating the coverage for Daedalus.
Below you can see the solar cycle of the years 1995-2019:
    
![Previous Years Solar Cycle](SolarCycle.png)

## Algorithm Description

1. Initialization
    - Define bins: every area of interest is defined by assigning values to its boundaries.
    - Load the Geomagnetic Kp indices into memory. User can select which years he wishes to load.
2. For each position of the orbit:
    - Read Magnetic Latitude, Magnetic Local Time and Altitude from the orbit file
    - Calculate Kp index according to past values.
    - Check if this satellite position falls into any of the predefined bins. 
      In case it does, count it as a hit and add to the time the satellite spends inside the bin.


## Plots
The data resulted after the execution of the current software are the duration the satellite spends inside each bin.
In order to visualize these four dimensional data we utilize several kind of plots.

### Bars
<img src="BarsPlot.png" alt="Bars Plot" style="width: 300px;"/>
The bars-plot contains one figure per Magnetic Latitude range. It visualizes all the bins as rectangles.
X axis represents the Magnetic Local Time and Kp index of the bin and y axis the Altitude.
Each rectangle is painted accordingly to what percent of the desired time the satellite stays inside the bin. 
(coverage information)

### Polar
<img src="PolarPlot.png" alt="Polar Plot" style="width: 300px;"/>
The polar-plot contains one figure per Altitude range. It visualizes the position of the bins in respect of 
the Magnetic Latitude (radial axis) and the Magnetic Local Time (angular axis).
The coverage information is displayed at the legend.

### Kp Scatter
<img src="ScatterPlot.png" alt="Kp Scatter Plot" style="width: 300px;"/>
The Kp-scatter-plot consists of a single figure. The x-axis represents the Magnetic Local Time and the
y-axis the Magnetic Latitude. The points are colored by their Kp value. The bins positions are depicted as
rectangles on the plot and the coverage information is displayed at the legend.



In [5]:
import sys
sys.path.insert(1, '../../SourceCode/')
import DaedalusGlobals as DaedalusGlobals

from os import path

import csv
import glob
import math
import time
import datetime
import numpy as np 
import pandas as pd
import ipywidgets as w

import plotly
import chart_studio.plotly as py 
import plotly.graph_objects as go
import matplotlib.cm

# Holds the Geomagnetic kp Indices. 
# Accessing examples: GeomagneticIndices[("23", "05", "2011", "0")] stores the kp index for 23-05-2001 00:00-03:00
#                     GeomagneticIndices[("23", "05", "2011", "3")] stores the kp index for 23-05-2001 09:00-12:00
GeomagneticIndices = dict()

# colors used at plotting
MyColors = ["#217ca3", "#e29930", "#919636", "#af1c1c", "#e7552c", "#1b4b5a", "#e4535e", "#aebd38", "#ffbb00", "#2c7873"]

# GUI elements with global scope
style1 = {'description_width':'170px'}
layout1 = {'width':'780px'}
style2 = {'description_width':'95px'}
layout2 = {'width':'160px'}
OrbitPreviewImage = w.Image( format='png', visible=False )
OrbitPreviewImage.layout.visibility = 'hidden'
ExecutionTitle_Text = w.Text(value="", description='Execution title:', style=style1, layout=layout1)
ExecutionDescr_Text = w.Text(value="", description='Execution description:', style=style1, layout=layout1)
OrbitFilenames_Dropdown = w.Dropdown( options=sorted(glob.glob(DaedalusGlobals.Orbit_Files_Path + "DAED_ORB_Lifetime*.csv")), description='Orbit files: ', style=style1, layout=layout1)
Years_Dropdown = w.Dropdown( value=2011, options=range(1932,2016),  description='Use Kp indices from year:', style=style1, layout=layout1)
SavedFilenames_Dropdown = w.Dropdown( options=sorted(glob.glob(DaedalusGlobals.CoverageResults_Files_Path + "*.CoverageResults.txt")),  description='', style=style1, layout=layout1)
PlotBars_Checkbox    = w.Checkbox(value=True,  description="Create Bars-Chart for each Magnetic Latitude", style=style1, layout=layout1 )
PlotPolar_Checkbox   = w.Checkbox(value=True,  description="Create Polar-Chart for each Altitude region", style=style1, layout=layout1 )
PlotKpScatter_Checkbox = w.Checkbox(value=False, description="Create Scatter-Chart of Kp-values for mission lifetime (Heavy)", style=style1, layout=layout1 )
PlotKpScatter_fromAlt  = w.IntText(value=0, description='From Alt.(km):', style=style2, layout=layout2)
PlotKpScatter_toAlt    = w.IntText(value=200, description='To Alt.(km):', style=style2, layout=layout2)

# Properties of the current coverage calculation
COVERAGE_Title = ""
COVERAGE_Description =""
COVERAGE_OrbitFilename = ""
COVERAGE_ResultsFilename = ""
COVERAGE_KpStartYear = ""


# utility: converts a number to its 2-digit string representation
def num_to_2digit_str( n ):
    s = str(n)
    if len(s) == 1:
        s = '0' + s
    return s

# utility: takes a string containing numbers and places spaces instead of the leading zeros 
def ConvertLeadingZerosToSpaces( str ):
    result = ""
    leading_zone = True
    for c in str:
        if leading_zone:
            if c == '0':
                result = result + ' '
            else:
                result = result + c
                leading_zone = False
        else:
            result = result + c
    if result.strip().startswith('.')  and  result.startswith(' '): result = result[:result.rfind(' ')] + '0' + result.strip()
    if result.strip() == "": result = result[ :-1 ] + '0'
    if (result.startswith('.')) : result = '0' + result            
    return result
            
        

# utility: returns a color of a colormap as list of r,g,b,a values representing a value inside a range
def getColor( Value, minValue, maxValue, ColormapName ):
    cmap = matplotlib.cm.get_cmap( ColormapName )
    norm = matplotlib.colors.Normalize(vmin=minValue, vmax=maxValue)
    rgba = cmap( norm(Value) )
    s = "rgba" + str(rgba) 
    return s

# Define a class which can describe a bin
class Bin:
    ID             = ""
    Description    = ""
    MLT_min        = 0 # Mean Local Time (hour & min of the 24-hour day) (string)
    MLT_max        = 0 # Mean Local Time (hour & min of the 24-hour day) (string)
    MagLat_min     = 0 # Magnetic Latitude (degrees)
    MagLat_max     = 0 # Magnetic Latitude (degrees)
    Altitude_min   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Altitude_max   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Kp_min         = 0 #
    Kp_max         = 0 #
    NumOfBins      = 0 # How many parts will the Altitude range be splitted in
    CumulativeTime = 0 # (sec)
    DesirableCumulativeTime = 0 # (sec)
    
    def __init__(self, ID, Description, MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Kp_min, Kp_max, DesirableCumulativeTime):
        self.ID             = ID
        self.Description    = Description
        self.MLT_min        = MLT_min 
        self.MLT_max        = MLT_max
        self.MagLat_min     = MagLat_min
        self.MagLat_max     = MagLat_max
        self.Altitude_min   = Altitude_min
        self.Altitude_max   = Altitude_max
        self.Kp_min         = Kp_min
        self.Kp_max         = Kp_max
        self.DesirableCumulativeTime = DesirableCumulativeTime
        
    def getInfo(self):
        s  = self.ID.ljust(8, ' ') + ": "
        s += "{:02.0f}".format(self.MLT_min)      + "<MLT<="    + "{:02.0f}".format(self.MLT_max)      + " "
        s += "{:03.0f}".format(self.MagLat_min)   + "<MagLat<=" + "{:03.0f}".format(self.MagLat_max)   + " "
        s += "{:03.0f}".format(self.Altitude_min) + "<Alt<="    + "{:03.0f}".format(self.Altitude_max) + " "
        s += str(self.Kp_min)             + "<Kp<="     + str(self.Kp_max)       + " "
        s += "Coverage=" 
        s += ConvertLeadingZerosToSpaces( "{:09.3f}".format(self.CumulativeTime/60) ) + "/" 
        s += ConvertLeadingZerosToSpaces( "{:06.2f}".format(self.DesirableCumulativeTime/60) ) + "min "
        s += ConvertLeadingZerosToSpaces( "{:08.0f}".format(self.CumulativeTime) ) + "/" 
        s += ConvertLeadingZerosToSpaces( "{:05.0f}".format(self.DesirableCumulativeTime) ) + "sec"
        return s
    
    def printMe(self):
        print( self.getInfo() )

# this list holds the definitions of all bins
Bins = list()
#                ID        Description                          MLT      MagLat    Altitude                Kp       DesiredTime(sec)
Bins.append( Bin("AEM_L1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115, 120,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L2", "Auroral E region, midnight sector", 22, 2,   60, 75,   120, 125,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L3", "Auroral E region, midnight sector", 22, 2,   60, 75,   125, 130,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L4", "Auroral E region, midnight sector", 22, 2,   60, 75,   130, 135,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L5", "Auroral E region, midnight sector", 22, 2,   60, 75,   135, 140,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_M1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115,       123.33333,   2, 4,   30*60 ) )
Bins.append( Bin("AEM_M2", "Auroral E region, midnight sector", 22, 2,   60, 75,   123.33333, 131.66666,   2, 4,   30*60 ) )
Bins.append( Bin("AEM_M3", "Auroral E region, midnight sector", 22, 2,   60, 75,   131.66666, 140,         2, 4,   30*60 ) )
Bins.append( Bin("AEM_H1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115, 140,               4, 9,   20*60 ) )

Bins.append( Bin("AFM_L1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 185,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L2", "Auroral F region, midnight sector", 22, 2,   60, 75,   185, 230,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L3", "Auroral F region, midnight sector", 22, 2,   60, 75,   230, 275,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L4", "Auroral F region, midnight sector", 22, 2,   60, 75,   275, 320,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L5", "Auroral F region, midnight sector", 22, 2,   60, 75,   320, 365,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L6", "Auroral F region, midnight sector", 22, 2,   60, 75,   365, 410,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L7", "Auroral F region, midnight sector", 22, 2,   60, 75,   410, 455,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L8", "Auroral F region, midnight sector", 22, 2,   60, 75,   455, 500,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_M1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 230,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_M2", "Auroral F region, midnight sector", 22, 2,   60, 75,   230, 320,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_M3", "Auroral F region, midnight sector", 22, 2,   60, 75,   320, 410,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_M4", "Auroral F region, midnight sector", 22, 2,   60, 75,   410, 500,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_H1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 260,               4, 9,   20*60 ) )
Bins.append( Bin("AFM_H2", "Auroral F region, midnight sector", 22, 2,   60, 75,   260, 380,               4, 9,   20*60 ) )
Bins.append( Bin("AFM_H3", "Auroral F region, midnight sector", 22, 2,   60, 75,   380, 500,               4, 9,   20*60 ) )

Bins.append( Bin("AEE_L1", "Auroral E region, evening sector",  16, 20,  60, 75,   115, 120,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L2", "Auroral E region, evening sector",  16, 20,  60, 75,   120, 125,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L3", "Auroral E region, evening sector",  16, 20,  60, 75,   125, 130,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L4", "Auroral E region, evening sector",  16, 20,  60, 75,   130, 135,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L5", "Auroral E region, evening sector",  16, 20,  60, 75,   135, 140,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_M1", "Auroral E region, evening sector",  16, 20,  60, 75,   115,       123.33333,   2, 4,   30*60 ) )
Bins.append( Bin("AEE_M2", "Auroral E region, evening sector",  16, 20,  60, 75,   123.33333, 131.66666,   2, 4,   30*60 ) )
Bins.append( Bin("AEE_M3", "Auroral E region, evening sector",  16, 20,  60, 75,   131.66666, 140,         2, 4,   30*60 ) )
Bins.append( Bin("AEE_H1", "Auroral E region, evening sector",  16, 20,  60, 75,   115, 140,               4, 9,   20*60 ) )

Bins.append( Bin("AED_L1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115, 120,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L2", "Auroral E region, dawn sector",     4, 8,  60, 75,   120, 125,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L3", "Auroral E region, dawn sector",     4, 8,  60, 75,   125, 130,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L4", "Auroral E region, dawn sector",     4, 8,  60, 75,   130, 135,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L5", "Auroral E region, dawn sector",     4, 8,  60, 75,   135, 140,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_M1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115,       123.33333,     2, 4,   30*60 ) )
Bins.append( Bin("AED_M2", "Auroral E region, dawn sector",     4, 8,  60, 75,   123.33333, 131.66666,     2, 4,   30*60 ) )
Bins.append( Bin("AED_M3", "Auroral E region, dawn sector",     4, 8,  60, 75,   131.66666, 140,           2, 4,   30*60 ) )
Bins.append( Bin("AED_H1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115, 140,                 4, 9,   20*60 ) )

Bins.append( Bin("EEJ_A1", "Equatorial E-region",             10, 13,  -7,  7,   115,       123.33333,     0, 9,   10*60 ) )
Bins.append( Bin("EEJ_A2", "Equatorial E-region",             10, 13,  -7,  7,   123.33333, 131.66666,     0, 9,   10*60 ) )
Bins.append( Bin("EEJ_A3", "Equatorial E-region",             10, 13,  -7,  7,   131.66666, 140,           0, 9,   10*60 ) )

Bins.append( Bin("EPB_A1", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   140, 185,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A2", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   185, 230,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A3", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   230, 275,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A4", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   275, 320,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A5", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   320, 365,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A6", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   365, 410,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A7", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   410, 455,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A8", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   455, 500,                  0, 9,   150*60 ) )

Bins.append( Bin("SQ_A1",  "Sq & midlat F region currents",    6, 19, -60, 60,   140, 185,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A2",  "Sq & midlat F region currents",    6, 19, -60, 60,   185, 230,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A3",  "Sq & midlat F region currents",    6, 19, -60, 60,   230, 275,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A4",  "Sq & midlat F region currents",    6, 19, -60, 60,   275, 320,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A5",  "Sq & midlat F region currents",    6, 19, -60, 60,   320, 365,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A6",  "Sq & midlat F region currents",    6, 19, -60, 60,   365, 410,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A7",  "Sq & midlat F region currents",    6, 19, -60, 60,   410, 455,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A8",  "Sq & midlat F region currents",    6, 19, -60, 60,   455, 500,                  0, 3,   150*60 ) )

Bins.append( Bin("CF_L1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 185,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L2", "Dayside Cusp F-region",            10, 14,   70,  80,   185, 230,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L3", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 275,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L4", "Dayside Cusp F-region",            10, 14,   70,  80,   275, 320,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L5", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 365,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L6", "Dayside Cusp F-region",            10, 14,   70,  80,   365, 410,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L7", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 455,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L8", "Dayside Cusp F-region",            10, 14,   70,  80,   455, 500,                0, 2,   50*60 ) )
Bins.append( Bin("CF_M1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 230,               2, 4,   30*60 ) )
Bins.append( Bin("CF_M2", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 320,               2, 4,   30*60 ) )
Bins.append( Bin("CF_M3", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 410,               2, 4,   30*60 ) )
Bins.append( Bin("CF_M4", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 500,               2, 4,   30*60 ) )
Bins.append( Bin("CF_H1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 260,               4, 9,   20*60 ) )
Bins.append( Bin("CF_H2", "Dayside Cusp F-region",            10, 14,   70,  80,   260, 380,               4, 9,   20*60 ) )
Bins.append( Bin("CF_H3", "Dayside Cusp F-region",            10, 14,   70,  80,   380, 500,               4, 9,   20*60 ) )

Bins.append( Bin("PCF_L1", "Polar cap F-region",              14, 10,   70,  90,   140, 185,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L2", "Polar cap F-region",              14, 10,   70,  90,   185, 230,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L3", "Polar cap F-region",              14, 10,   70,  90,   230, 275,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L4", "Polar cap F-region",              14, 10,   70,  90,   275, 320,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L5", "Polar cap F-region",              14, 10,   70,  90,   320, 365,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L6", "Polar cap F-region",              14, 10,   70,  90,   365, 410,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L7", "Polar cap F-region",              14, 10,   70,  90,   410, 455,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L8", "Polar cap F-region",              14, 10,   70,  90,   455, 500,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_M1", "Polar cap F-region",              14, 10,   70,  90,   140, 230,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_M2", "Polar cap F-region",              14, 10,   70,  90,   230, 320,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_M3", "Polar cap F-region",              14, 10,   70,  90,   320, 410,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_M4", "Polar cap F-region",              14, 10,   70,  90,   410, 500,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_H1", "Polar cap F-region",              14, 10,   70,  90,   140, 260,               4, 9,   20*60 ) )
Bins.append( Bin("PCF_H2", "Polar cap F-region",              14, 10,   70,  90,   260, 380,               4, 9,   20*60 ) )
Bins.append( Bin("PCF_H3", "Polar cap F-region",              14, 10,   70,  90,   380, 500,               4, 9,   20*60 ) )



def createBinsForTheWholeEarth():
    n = 1
    for Kp_min in [0, 2, 4]:
        if Kp_min == 0:
            Kp_max = 2
        elif Kp_min == 2:
            Kp_max = 4
        elif Kp_min == 4:
            Kp_max = 9
        ####
        for MLT_min in range(0, 24, 4):
            for MagLat_min in range(-180, 180, 20):
                for Alt_min in range(115, 250, 25):
                    n = n + 1
                    Bins.append( Bin("E"+str(n), "",            MLT_min, MLT_min+4,   MagLat_min, MagLat_min+15,   Alt_min, Alt_min+25,               Kp_min, Kp_max,   20*60 ) )                    
    print ( len(Bins) + " Bins covering the whole Earth.")    
#createBinsForTheWholeEarth()

# cheks if certain MLT lies in a certain range. Created in order to take account ranges like 22-2
def is_MLT_inside_range( MLT, MLT_min, MLT_max ):
    if MLT_min <= MLT_max: # example: from 13 to 18 hour
        return (MLT > MLT_min  and  MLT <= MLT_max)
    else: # example: from 22 to 3 hour
        return (MLT > MLT_min  or   MLT <= MLT_max)

# returns: the bin object which matches the arguments
def GetMatchedBin( MLT, MagLat, Altitude, Kp ):
    MatchedBin = None
    for B in Bins:
        Kp_min_to_check = B.Kp_min
        if Kp_min_to_check == 0: Kp_min_to_check = -1
        ####
        if is_MLT_inside_range(MLT, B.MLT_min, B.MLT_max):
            if         MagLat   > B.MagLat_min    and  MagLat   <= B.MagLat_max:
                if     Altitude > B.Altitude_min  and  Altitude <= B.Altitude_max:
                    if Kp       > Kp_min_to_check and  Kp       <= B.Kp_max:
                        MatchedBin = B
                        break
    return MatchedBin


# returns: the bin object which matches the arguments
def getBinByItsProperties( MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Kp_min, Kp_max ):
    CorrectBin = None
    for B in Bins:
        if             MLT_min      == B.MLT_min       and  MLT_max      == B.MLT_max:
            if         MagLat_min   == B.MagLat_min    and  MagLat_max   == B.MagLat_max:
                if     Altitude_min == B.Altitude_min  and  Altitude_max == B.Altitude_max:
                    if Kp_min       == B.Kp_min        and  Kp_max       == B.Kp_max:
                        CorrectBin = B
                        break
    return CorrectBin


# Reads the Geomagnetic kp Indices from text files and stores them in a dictionary.
# Data Source (explanation): https://www.ngdc.noaa.gov/stp/GEOMAG/kp_ap.html
# Data Source (download)   : ftp://ftp.ngdc.noaa.gov/STP/GEOMAGNETIC_DATA/INDICES/KP_AP
def readGeomagneticIndices(fromYear, toYear):
    global GeomagneticIndices
    for Y in range(fromYear, toYear): # this range should be small for execution speed
        with open(DaedalusGlobals.GeomagneticIndices_Files_Path + str(Y)) as fp:
            for line in fp:
                year  = "20" + line[0:2]
                month = line[2:4]
                day   = line[4:6]
                kp00  = float(line[12:14]) / 10
                kp03  = float(line[14:16]) / 10
                kp06  = float(line[16:18]) / 10
                kp09  = float(line[18:20]) / 10
                kp12  = float(line[20:22]) / 10
                kp15  = float(line[22:24]) / 10
                kp18  = float(line[24:26]) / 10
                kp21  = float(line[26:28]) / 10
                GeomagneticIndices[(day, month, year, "0")] = kp00
                GeomagneticIndices[(day, month, year, "1")] = kp03
                GeomagneticIndices[(day, month, year, "2")] = kp06
                GeomagneticIndices[(day, month, year, "3")] = kp09
                GeomagneticIndices[(day, month, year, "4")] = kp12
                GeomagneticIndices[(day, month, year, "5")] = kp15
                GeomagneticIndices[(day, month, year, "6")] = kp18
                GeomagneticIndices[(day, month, year, "7")] = kp21
            fp.close()
        
# returns: 1. how many bin misses occured
#          2. how many bin hits occured
#          3. the filename where the coverage results are stored
#          4. the duration of execution in seconds
def CalculateCoverage():
    SelectedOrbit_OnlyFilename = OrbitFilenames_Dropdown.value[OrbitFilenames_Dropdown.value.rfind("/")+1:OrbitFilenames_Dropdown.value.rfind(".")]
    ResultsFilename = DaedalusGlobals.CoverageResults_Files_Path + SelectedOrbit_OnlyFilename + ".CoverageResults.txt"    
    # init the properties of the current coverage calculation
    global COVERAGE_Title, COVERAGE_Description, COVERAGE_OrbitFilename, COVERAGE_ResultsFilename, COVERAGE_KpStartYear
    COVERAGE_Title           = ExecutionTitle_Text.value
    COVERAGE_Description     = ExecutionDescr_Text.value
    COVERAGE_OrbitFilename   = OrbitFilenames_Dropdown.value
    COVERAGE_ResultsFilename = ResultsFilename
    COVERAGE_KpStartYear     = str(Years_Dropdown.value)
    ####
    Kp = 0
    startSecs = time.time()
    BinMisses = BinHits = 0
    PREV_time  = CURR_time  = None
    PREV_BinID = CURR_BinID = ""
    if path.exists( ResultsFilename ):
        print( "File " + ResultsFilename + " already exists. Cannot continue in order to prevent overwriting useful data." )
        return 0, 0, "", 0 # <<<<
    ########
    with open( COVERAGE_OrbitFilename ) as CSVfile:        
        CSVreader = csv.reader( CSVfile )
        # locate the column numnbers of interest inside the csv file
        CSVheader = next( CSVreader )
        Time_idx     = CSVheader.index( "Time (UTCG)" ) #CSVheader.index( "Daedalus.EpochText" )
        Lat_idx      = CSVheader.index( "Lat (deg)" ) #CSVheader.index( "Daedalus.Latitude" )
        Lon_idx      = CSVheader.index( "Lon (deg)" ) #CSVheader.index( "Daedalus.Longitude" )
        Altitude_idx = CSVheader.index( "Alt (km)" ) #CSVheader.index( "Daedalus.Height" )
        MagLat_idx   = CSVheader.index( "Daedalus.Magnetic Latitude" )
        MLT_idx      = CSVheader.index( "Daedalus.MLT" )
        # read the satellite positions and try to fill the bins
        n = 0
        for row in CSVreader: # for each satellite position
            n = n + 1
            PREV_time = CURR_time
            # parse the date-time of this satellite position
            CURR_time = datetime.datetime.strptime(row[Time_idx], '%d %b %Y %H:%M:%S.%f')
            year  = CURR_time.year
            month = CURR_time.month
            day   = CURR_time.day
            hour  = CURR_time.hour
            # calculate the Kp index for this particular time
            try:
                Kp = GeomagneticIndices[(num_to_2digit_str(day), num_to_2digit_str(month), num_to_2digit_str(Years_Dropdown.value+year-2028), str(hour//3))]
            except:
                if month==2 and day==29: # the leap years may correspond to non-leap years at the selected range of years for Kp calculation
                    Kp = GeomagneticIndices[(num_to_2digit_str(28), num_to_2digit_str(month), num_to_2digit_str(Years_Dropdown.value+year-2028), str(hour//3))]
            # remember some useful properties of this satellite position
            MLT      = float( row[MLT_idx] )
            MagLat   = float( row[MagLat_idx] )
            Altitude = float( row[Altitude_idx] )
            # Check if the satellite position can be assigned to a bin
            mathedBin = GetMatchedBin( MLT, MagLat, Altitude, Kp )
            # If the satellite is inside a bin during the last 2 positions then calculate the duration
            PREV_BinID = CURR_BinID
            if mathedBin is None:
                CURR_BinID = ""
            else:
                CURR_BinID = mathedBin.ID            
            if len(CURR_BinID) > 0  and  CURR_BinID == PREV_BinID:
                BinHits = BinHits + 1
                DurationInsideBin = (CURR_time - PREV_time).seconds
                mathedBin.CumulativeTime += DurationInsideBin 
            elif len(CURR_BinID) > 0:
                BinMisses = BinMisses + 1
    # calculate duration of execution
    finishSecs = time.time()
    DurationOfExecution = finishSecs-startSecs
    # Save the results in a text file
    nowstr = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")    
    F = open(ResultsFilename, 'w')
    F.write( "# -- COVERAGE RESULTS -- " + "\n"  )
    F.write( "# Date of execution: " + nowstr + "\n" )
    F.write( "# Title: " + COVERAGE_Title + "\n" )
    F.write( "# Description: " + COVERAGE_Description + "\n")
    F.write( "# Parsed " + str(n) + " lines from orbit file " + COVERAGE_OrbitFilename + "\n")
    F.write( "# Used Kp indices starting from year " + COVERAGE_KpStartYear + "\n")
    F.write( "# Bin Misses: " + str(BinMisses) + "     Bin Hits: " + str(BinHits) + "\n")
    F.write( "# Duration of execution: " + ConvertLeadingZerosToSpaces("{0:.0f}".format(DurationOfExecution)) + " seconds  or  " + ConvertLeadingZerosToSpaces("{0:.2f}".format(DurationOfExecution/60))  + " minutes" + "\n" )
    F.write( "# " + "\n")    
    for B in Bins:
        F.write( B.getInfo() + "\n" )
    F.close()
    #
    return BinMisses, BinHits, ResultsFilename, DurationOfExecution

        

#################### EVENT LISTENERS ###########################
def Exec_Btn_Clicked( b ):
    print( "Coverage Calculation started." )
    readGeomagneticIndices( Years_Dropdown.value, Years_Dropdown.value+4 )
    BinMisses, BinHits, ResultsFilename, Duration = CalculateCoverage()
    if len(ResultsFilename) > 0:
        print( "Coverage Calculation finshed in " + str(Duration) + " seconds, with " + str(BinMisses) + " bin misses, " + str(BinHits) + " bin hits." )
        print( "RESULTS (stored in " + ResultsFilename + "):" )
        for B in Bins:
            B.printMe()
        if PlotBars_Checkbox.value      == True: 
            print("Ploting Bin data on a bar-chart")
            PlotBins_Rectangles()
        if PlotPolar_Checkbox.value     == True: 
            print("Ploting Bin data on a polar-chart")
            PlotBins_Polar()
        if PlotKpScatter_Checkbox.value == True: 
            print("Ploting Bin data on a Kp scatter-chart")
            PlotBins_KpScatter(OrbitFilenames_Dropdown.value, int(Years_Dropdown.value), PlotKpScatter_fromAlt.value, PlotKpScatter_toAlt.value )
def Load_Btn_Clicked( b ):
    global COVERAGE_Title, COVERAGE_Description, COVERAGE_OrbitFilename, COVERAGE_ResultsFilename, COVERAGE_KpStartYear
    if len(SavedFilenames_Dropdown.value) > 0:
        COVERAGE_ResultsFilename = ResultsFilename = SavedFilenames_Dropdown.value
        with open(COVERAGE_ResultsFilename, 'r') as F:
            for line in F:
                if line[0:1] == '#': # this line contains a comment, print it as it is.
                    print ( line[1:len(line)-1] )
                    if line.startswith("# Title:"): COVERAGE_Title = line[8:].strip()
                    if line.startswith("# Description:"): COVERAGE_Description = line[14:].strip()
                    if line.startswith("# Parsed"): COVERAGE_OrbitFilename = line[line.find("orbit file")+11:].strip()
                    if line.startswith("# Used Kp"): COVERAGE_KpStartYear = line[37:].strip()
                else: # this line contains bin info, print it and store them in the correct bin.
                    print ( line[:len(line)-1] )
                    aBinID = line[:line.find(":")].strip()
                    secondsInBin = float( line[line.rfind("min")+3:line.rfind("/")] )
                    for B in Bins:
                        if B.ID == aBinID:
                            B.CumulativeTime = secondsInBin
                            break
        F.close()
        ##
        readGeomagneticIndices( int(COVERAGE_KpStartYear), int(COVERAGE_KpStartYear)+4)
        if PlotBars_Checkbox.value      == True: 
            print("Ploting Bin data on a bar-chart")
            PlotBins_Rectangles()
        if PlotPolar_Checkbox.value     == True:
            print("Ploting Bin data on a polar-chart")
            PlotBins_Polar()
        if PlotKpScatter_Checkbox.value == True: 
            print("Ploting Bin data on a Kp scatter-chart")
            PlotBins_KpScatter(COVERAGE_OrbitFilename, int(COVERAGE_KpStartYear), PlotKpScatter_fromAlt.value, PlotKpScatter_toAlt.value)
            
################################################################

def OrbitFilenames_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        OrbitFilename = change['new']
        ImageFileName = OrbitFilename[:-4] + ".png"
        if path.exists( ImageFileName ):
            OrbitPreviewImage.value = open(ImageFileName, "rb").read()
            OrbitPreviewImage.visible = True
            OrbitPreviewImage.layout.visibility = 'visible'
        else:
            OrbitPreviewImage.visible = False
            OrbitPreviewImage.layout.visibility = 'hidden'

def SavedFilenames_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        OrbitFilename = ""
        # read results file and find the orbit filename
        with open( change['new'], 'r') as F:
            for line in F:
                if line.startswith("# Parsed"): 
                    OrbitFilename = line[line.find("orbit file")+11:].strip()
                    break
        F.close()
        # load image
        ImageFileName = OrbitFilename[:-4] + ".png"
        if path.exists( ImageFileName ):
            OrbitPreviewImage.value = open(ImageFileName, "rb").read()
            OrbitPreviewImage.visible = True
            OrbitPreviewImage.layout.visibility = 'visible'
        else:
            OrbitPreviewImage.visible = False
            OrbitPreviewImage.layout.visibility = 'hidden'

def createGUI():
    ## the top level visual elements
    MainPanel = w.VBox()    
    MainTab = w.Tab() 
    CalcCoveragePanel = w.VBox()
    LoadCoveragePanel = w.VBox()
    ## the checkboxes which allow user to select which plots he wants to create
    PlotSelectionPanel = w.VBox()
    PlotSelectionPanel.children = [PlotBars_Checkbox, PlotPolar_Checkbox, w.HBox([PlotKpScatter_Checkbox,PlotKpScatter_fromAlt,PlotKpScatter_toAlt])]
    ##
    MainTab.children = [ CalcCoveragePanel,    LoadCoveragePanel ]
    MainTab.set_title(0, 'Calculate Coverage')
    MainTab.set_title(1, 'Load Coverage')
    MainPanel.children = [ OrbitPreviewImage, MainTab ]    
    ## 
    Exec_Btn = w.Button (description='Calculate Coverage',tooltip="Click here to calculate the Daedalus coverage results",)
    Exec_Btn.style.button_color = 'MediumTurquoise'
    Exec_Btn.on_click( Exec_Btn_Clicked )
    CalcCoveragePanel.children = [OrbitFilenames_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, Years_Dropdown, PlotSelectionPanel, Exec_Btn ]
    ##
    Load_Btn = w.Button (description='Load Coverage from',tooltip="Click here to plot the Daedalus coverage results",)
    Load_Btn.style.button_color = 'YellowGreen'
    Load_Btn.on_click( Load_Btn_Clicked )
    L2_horizontal = w.HBox()
    L2_horizontal.children = [Load_Btn, SavedFilenames_Dropdown]
    LoadCoveragePanel.children = [ w.HBox([Load_Btn, SavedFilenames_Dropdown]), PlotSelectionPanel ]
    ## Assign event listeners
    OrbitFilenames_Dropdown.observe( OrbitFilenames_Dropdown_onChange )
    SavedFilenames_Dropdown.observe( SavedFilenames_Dropdown_onChange )
    return MainPanel

    


def PlotBins_Rectangles():
    ColumnWidth = 80
            
    All_MagLatRanges = list()
    for B in Bins:
        if [B.MagLat_min, B.MagLat_max] not in All_MagLatRanges:
            All_MagLatRanges.append( [B.MagLat_min, B.MagLat_max] )            
    
    for aMagLatRange in All_MagLatRanges:
        # We will create one plot for each MagLatRange 
        fig = go.Figure()
        # init
        Xposition = 20
        XaxisTickPositions = list()
        XaxisTickLabels = list()
        TheLowestAltitudeInThePlot  = 10000
        TheHighestAltitudeInThePlot = 0
        
        # find out all MLT ranges for this MagLat range
        All_MLTRanges = list()
        tickValues = list()
        for B in Bins:
            if B.MagLat_min==aMagLatRange[0] and  B.MagLat_max==aMagLatRange[1]:
                # remember all Latitudes in order to display them as tick values on the y axis
                if B.Altitude_min not in tickValues: tickValues.append( B.Altitude_min )
                if B.Altitude_max not in tickValues: tickValues.append( B.Altitude_max )
                #
                if [B.MLT_min, B.MLT_max] not in All_MLTRanges:
                    All_MLTRanges.append( [B.MLT_min, B.MLT_max] )
        
        # find out all Kp ranges for this MagLat range and this MLTrange 
        for aMLTrange in All_MLTRanges:
            All_KpRanges = list()
            for B in Bins:
                if B.MagLat_min==aMagLatRange[0] and  B.MagLat_max==aMagLatRange[1]:
                    if B.MLT_min==aMLTrange[0] and  B.MLT_max==aMLTrange[1]:
                        if [B.Kp_min, B.Kp_max] not in All_KpRanges:
                            All_KpRanges.append( [B.Kp_min, B.Kp_max] )

            # find out all Altitude ranges for this MagLat range, this MLT range and this Kp range            
            for aKpRange in All_KpRanges:
                XaxisTickPositions.append( Xposition + ColumnWidth/2 )
                XaxisTickLabels.append( "Kp " + str(aKpRange[0]) + "-" + str(aKpRange[1]) + "<br>" + str(aMLTrange[0]) + "-" + str(aMLTrange[1]) )
                ####
                All_AltitudeRanges = list()
                for B in Bins:
                    if B.MagLat_min==aMagLatRange[0] and  B.MagLat_max==aMagLatRange[1]:
                        if B.MLT_min==aMLTrange[0] and  B.MLT_max==aMLTrange[1]:
                            if B.Kp_min==aKpRange[0] and  B.Kp_max==aKpRange[1]:
                                All_AltitudeRanges.append( [B.Altitude_min, B.Altitude_max] )
                                # calculate the range of the Y axis
                                if B.Altitude_min<TheLowestAltitudeInThePlot: TheLowestAltitudeInThePlot=B.Altitude_min
                                if B.Altitude_max>TheHighestAltitudeInThePlot: TheHighestAltitudeInThePlot=B.Altitude_max
            
                # create one figure per altitude range
                for anAltitudeRange in All_AltitudeRanges:
                    currentBin = getBinByItsProperties( aMLTrange[0], aMLTrange[1], aMagLatRange[0], aMagLatRange[1], anAltitudeRange[0], anAltitudeRange[1], aKpRange[0], aKpRange[1] )
                    fig.add_shape(go.layout.Shape(
                        type="rect", xref="x", yref="y", 
                        x0=Xposition, y0=currentBin.Altitude_min, x1=Xposition+ColumnWidth, y1=currentBin.Altitude_max,
                        line=dict( color="RoyalBlue", width=3,), fillcolor=getColor(currentBin.CumulativeTime, 0, currentBin.DesirableCumulativeTime, "BuPu"), opacity=0.8, 
                    ))
                    BinTitle = currentBin.ID + "<br>" + "{:3.0f}".format(currentBin.CumulativeTime/60) +  " / " + "{:2.0f}".format(currentBin.DesirableCumulativeTime/60)
                    fig.add_trace(go.Scatter(x=[Xposition + ColumnWidth/2],y=[(currentBin.Altitude_max+currentBin.Altitude_min)/2],text=[BinTitle],mode="text", ))
                Xposition += ColumnWidth + 10
            Xposition += 28
        
        FigureTitle = ""
        if COVERAGE_Title.strip() != "": FigureTitle += COVERAGE_Title + "  -- "
        FigureTitle += "Kp indices start from year " + COVERAGE_KpStartYear + "<br>"
        FigureTitle += "Orbit file:" + COVERAGE_OrbitFilename[COVERAGE_OrbitFilename.rfind('/')+1:] + "<br>" 
        FigureTitle += "Results file:" + COVERAGE_ResultsFilename[COVERAGE_ResultsFilename.rfind('/')+1:] + "<br>"
        FigureTitle += "<b>Magnetic Latitudes from " + str(aMagLatRange[0]) + "&#176; to " + str(aMagLatRange[1]) + "&#176;" + "</b>" + "<br>"
        #FigureTitle += "(boxes contain minutes of cummulative time the satellite spends inside each bin)"
        fig.update_layout(width=1100, height=1640, showlegend=False, title=FigureTitle,
                          xaxis_title="Kp index range<br>Magnetic Local Time range", yaxis_title="Altitude (km)", 
                          margin=go.layout.Margin(b=150,t=150), 
                          colorscale=dict(sequential='Blues') ) 
        # add a colorbar
        fig.add_trace(go.Scatter( x=[0,0], y=[0,0], opacity=0,  mode="markers",
            marker=dict(colorscale="Blues", color=[0,100], colorbar=dict(tickvals=[0,100], ticktext=['empty','full'] ),),
        ))
        
        fig.update_xaxes(range=[0, Xposition-10], showgrid=False)
        fig.update_xaxes( tickmode = 'array', tickvals=XaxisTickPositions,  ticktext=XaxisTickLabels )
        #fig.update_yaxes(type="log", range=[2.055, 2.71], tickvals=tickValues ) 
        fig.update_yaxes(type="log", range=[ math.log(TheLowestAltitudeInThePlot,10)-0.004, math.log(TheHighestAltitudeInThePlot,10)+0.008], tickvals=tickValues ) 
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig) 
        
        
        
        
        
def PlotBins_Polar():
    PlotBackgroundColor = "Gainsboro"
    MyColorsIndex = 0
    AltitudeREGIONS = [ [115,140], [140,500] ]
    
    for anAltitudeREGION in AltitudeREGIONS:    
        MyColorsIndex = 0
        # Find out all Magnetic Latitude Values which will be displayed in the plot, at the radial axis
        AllMagLatValues = list()
        for B in Bins:
            if B.Altitude_min>=anAltitudeREGION[0] and B.Altitude_max<=anAltitudeREGION[1]:
                if B.MagLat_min not in AllMagLatValues: AllMagLatValues.append(B.MagLat_min)
                if B.MagLat_max not in AllMagLatValues: AllMagLatValues.append(B.MagLat_max)
                
        # for each bin which belongs to the region
        fig = go.Figure()
        for B in Bins:
            if B.Altitude_min>=anAltitudeREGION[0] and B.Altitude_max<=anAltitudeREGION[1]:
                # convert bin values in order to be displayed on the polar plot
                MLTmin = B.MLT_min
                MLTmax = B.MLT_max
                if MLTmin > MLTmax: MLTmax += 24 # takes account of durations like 22:00-02:00
                SliceAngle  = 15*(MLTmin+(MLTmax - MLTmin)/2) -90 # 360degrees/24hours=15
                SliceWidth  = 15*(MLTmax - MLTmin)
                # construct the bin info string
                BinInfo = "{:3.0f}".format(B.Altitude_min) + "-" + "{:3.0f}".format(B.Altitude_max) + "km " + ": Kp"+ str(B.Kp_min) + "-" + str(B.Kp_max) + ": " 
                if B.CumulativeTime > B.DesirableCumulativeTime: BinInfo += "<b>"
                BinInfo += "{:3.0f}".format(B.CumulativeTime/60) 
                if B.CumulativeTime >= B.DesirableCumulativeTime: BinInfo += "</b>"
                BinInfo += "/" + "{:3.0f}".format(B.DesirableCumulativeTime/60)  + "min"
                    
                # check if this slice has been already plotted
                FoundFigureIndex = -1
                for i in range( len(fig.data) ):
                    if fig.data[i]["theta"][0]==SliceAngle and fig.data[i]["width"][0]==SliceWidth:
                        FoundFigureIndex = i
                        break

                # plot a new slice for this bin or add info about this bin to the legend text
                if FoundFigureIndex >= 0:
                    fig.data[ FoundFigureIndex ][ "name" ] += BinInfo + "<br>"
                else:
                    fig.add_trace(go.Barpolar( base=[B.MagLat_min], r=[B.MagLat_max-B.MagLat_min], theta=[SliceAngle], width=[SliceWidth], 
                        text=[B.ID], marker_color=[MyColors[MyColorsIndex]],   opacity=0.84,
                        name = "<b>" + B.Description + "</b>" + "<br>" + BinInfo + "<br>"
                    ))
                    MyColorsIndex += 1
                    if MyColorsIndex >= len(MyColors): MyColorsIndex = 0
            
        # construct the radial axis values
        if 90 not in AllMagLatValues: AllMagLatValues.append(90)
        RadialAxisTickValues= list()
        for i in range( len(AllMagLatValues) ):
            RadialAxisTickValues.append( str(AllMagLatValues[i]) + "&#176;" )
        
        # Construct the plot's title
        FigureTitle = ""
        if COVERAGE_Title.strip() != "": FigureTitle += COVERAGE_Title + "  -- "
        FigureTitle += "Orbit file:" + COVERAGE_OrbitFilename[COVERAGE_OrbitFilename.rfind('/')+1:] + "<br>" 
        FigureTitle += "Results file:" + COVERAGE_ResultsFilename[COVERAGE_ResultsFilename.rfind('/')+1:] + "<br>"
        FigureTitle += "<b>Altitude from " + str(anAltitudeREGION[0]) + " to " + str(anAltitudeREGION[1]) + "</b>" + "<br>"
        
        # define the plot's layout
        fig.update_layout(width=940, height=900, showlegend=True, title=FigureTitle,
                          polar = dict(
                            bgcolor=PlotBackgroundColor,
                            radialaxis  =  dict(range = [90, min(AllMagLatValues)], tickvals=AllMagLatValues, ticktext=RadialAxisTickValues, tickangle=90, categoryorder = "category descending" ), 
                            angularaxis = dict(tickvals=[      0,      30,      60,      90,     120,     150,     180,     210,     240,     270,     300,     330], 
                                               ticktext=['06:00', '08:00', '10:00', '12:00', '14:00', '16:00', '18:00', '20:00', '22:00', '00:00', '02:00', '04:00'] )
                         ),
                         margin=go.layout.Margin(b=150,t=150), )
        # plot it
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig) 
        
        
def PlotBins_KpScatter(OrbitFileName, kp_startYear, from_Altitude, to_Altitude):
    StartSecs = time.time()
    #print( "START", StartSecs )
    
    with open( OrbitFileName ) as CSVfile:
        CSVreader = csv.reader( CSVfile )
        # locate the column numnbers of interest inside the csv file
        CSVheader = next( CSVreader )
        Time_idx     = CSVheader.index( "Time (UTCG)" ) #CSVheader.index( "Daedalus.EpochText" )
        Lat_idx      = CSVheader.index( "Lat (deg)" ) #CSVheader.index( "Daedalus.Latitude" )
        Lon_idx      = CSVheader.index( "Lon (deg)" ) #CSVheader.index( "Daedalus.Longitude" )
        Altitude_idx = CSVheader.index( "Alt (km)" ) #CSVheader.index( "Daedalus.Height" )
        MagLat_idx   = CSVheader.index( "Daedalus.Magnetic Latitude" )
        MLT_idx      = CSVheader.index( "Daedalus.MLT" )
        # read the satellite positions and add them to lists for ploting
        new_x_axis = list()
        new_y_axis = list()
        kp_array   = list()
        for row in CSVreader: # for each satellite position
            CURR_Altitude = float( row[Altitude_idx] )
            if CURR_Altitude >= from_Altitude  and  CURR_Altitude <= to_Altitude:
                CURR_time = datetime.datetime.strptime(row[Time_idx], '%d %b %Y %H:%M:%S.%f')
                year  = CURR_time.year
                month = CURR_time.month
                day   = CURR_time.day
                hour  = CURR_time.hour
                try: # calculate the Kp index for this particular time
                    Kp = GeomagneticIndices[(num_to_2digit_str(day), num_to_2digit_str(month), num_to_2digit_str(Years_Dropdown.value+year-2028), str(hour//3))]
                except:
                    if month==2 and day==29: # the leap years may correspond to non-leap years at the selected range of years for Kp calculation
                        Kp = GeomagneticIndices[(num_to_2digit_str(28), num_to_2digit_str(month), num_to_2digit_str(Years_Dropdown.value+year-2028), str(hour//3))]
                # remember some useful properties of this satellite position
                new_x_axis.append( float( row[MLT_idx]    ) )
                new_y_axis.append( float( row[MagLat_idx] ) )
                kp_array.append  ( Kp )
        
    #print( "DATA", time.time()-StartSecs )
    
    #df['Daedalus.MLT'] = df['Daedalus.MLT'][0::50]
    #df['Daedalus.Magnetic Latidude'] = df['Daedalus.Magnetic Latidude'][0::50]
    #df['Kp'] = df['Kp'][0::50]
    
    # create the Kp scatter 
    fig = go.Figure()
    fig.add_trace(go.Scatter( x=new_x_axis, y=new_y_axis,
                              mode='markers', marker=dict(color=kp_array, cmin=0, cmax=9, size=3, colorscale='rainbow',showscale=True, 
                                                          colorbar=dict(title="Kp", xanchor="left", x=-0.26, tick0=0, dtick=1, tickvals=[0,1,2,3,4,5,6,7,8,9], ticks="inside") ) ))
    fig.update_layout( width=1200, height=800, showlegend=False, coloraxis_showscale=False, title="Orbit file:" + OrbitFileName + "<br>" + "Kp values start year is " + str(kp_startYear) + "<br>" + "<b>Kp Indices during Mission Lifetime for Altitudes: "+str(from_Altitude)+"-"+str(to_Altitude)+" km</b>")
    fig.update_xaxes(title="Magnetic Local Time", showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig.update_layout(xaxis = dict(tickmode = 'linear',tick0 = 0,dtick = 2))    
    fig.update_yaxes(title="Magnetic Latitude (deg)", showgrid=True, gridwidth=0.5, gridcolor='gray') 
    fig.update_layout(yaxis = dict(tickmode = 'linear',tick0 = -90,dtick =30), margin=go.layout.Margin(b=150,t=150), width=1000, height=800, showlegend=True)
    fig.update_xaxes(range=[0, 24],  showline=True, linewidth=2, linecolor='gray', mirror=True)
    fig.update_yaxes(range=[-90, 90],showline=True, linewidth=2, linecolor='gray', mirror=True)
    # draw rectangles to represent the bins on the figure        
    MyColorsIndex = 0
    for B in Bins:
        if B.Altitude_min>=from_Altitude and B.Altitude_max<=to_Altitude:
            BinInfo = "{:3.0f}".format(B.Altitude_min) + "-" + "{:3.0f}".format(B.Altitude_max) + "km " + ": Kp"+ str(B.Kp_min) + "-" + str(B.Kp_max) + ": " 
            if B.CumulativeTime > B.DesirableCumulativeTime: BinInfo += "<b>"
            BinInfo += "{:3.0f}".format(B.CumulativeTime/60) 
            if B.CumulativeTime > B.DesirableCumulativeTime: BinInfo += "</b>"
            BinInfo += "/" + "{:3.0f}".format(B.DesirableCumulativeTime/60)  + "min"
            if B.MLT_min <= B.MLT_max: # ex: 10:00-14:00
                Xmin1 = B.MLT_min
                Xmax1 = B.MLT_max
                Xmin2 = -1
                Xmax2 = -1

            else: # ex: 22:00-02:00
                Xmin1 = B.MLT_min
                Xmax1 = 24
                Xmin2 = 0
                Xmax2 = B.MLT_max
            # check if this rectangle has been already plotted
            FoundFigureIndex = -1 
            for i in range( len(fig.layout['shapes']) ):
                if fig.layout['shapes'][i]["x0"]==Xmin1 and fig.layout['shapes'][i]["y0"]==B.MagLat_min and fig.layout['shapes'][i]["x1"]==Xmax1 and fig.layout['shapes'][i]["y1"]==B.MagLat_max:
                    FoundFigureIndex = i
                    break
            # add info about this bin to the legend text and/or draw a new rectangle
            if FoundFigureIndex >= 0: 
                for i in range( len(fig['data']) ):
                    if fig['data'][i]['name'] is not None  and   B.Description in fig['data'][i]['name']:
                        fig['data'][i]['name'] += BinInfo + "<br>"
            else:
                # draw a rectangle representing the bin
                fig.add_shape(go.layout.Shape( type="rect", xref="x", yref="y", opacity=0.7, layer="above",
                    x0=Xmin1, y0=B.MagLat_min, x1=Xmax1, y1=B.MagLat_max, 
                    line=dict(color=MyColors[MyColorsIndex], width=2,), fillcolor=MyColors[MyColorsIndex], 
                ))
                # add a trace so that a legend about the rectangle is created
                fig.add_trace(go.Scatter( x=[0], y=[0], marker=dict(color=MyColors[MyColorsIndex], size=0, opacity=0), 
                              name="<b>" + B.Description + "</b>" + "<br>" + BinInfo + "<br>" ))
                # draw a 2nd rectangle if necessary for this bin (when for ex it is 22:00-02:00)
                if Xmin1>=0 and Xmin2>=0:
                    fig.add_shape(go.layout.Shape( type="rect", xref="x", yref="y", opacity=0.7, layer="above",
                        x0=Xmin2, y0=B.MagLat_min, x1=Xmax2, y1=B.MagLat_max, 
                        line=dict(color=MyColors[MyColorsIndex], width=2,), fillcolor=MyColors[MyColorsIndex],
                    ))
                MyColorsIndex += 1
                
    # plot it
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig) 
    #print( "FIGURE", time.time()-StartSecs )

        
display( createGUI() )
